In [ ]:
from collections import defaultdict
import ee
import pandas as pd
import geopandas as gpd
import geemap

In [ ]:
ee.Authenticate()

In [2]:
ee.Initialize()

In [ ]:
# Global warming potentials from IPCC AR5 Table 8.7. Using GWP-100.
# https://www.ipcc.ch/site/assets/uploads/2018/02/WG1AR5_Chapter08_FINAL.pdf
GWP = {
    'CO2': 1,
    'N2O': 265,
    'CH4': 28
}

In [ ]:
ch4_ic = ee.ImageCollection('projects/wri-datalab/SCL-Cities/EDGAR-GHG/CH4')
co2_exclshort_ic = ee.ImageCollection('projects/wri-datalab/SCL-Cities/EDGAR-GHG/CO2exorgshort')
co2_short_ic = ee.ImageCollection('projects/wri-datalab/SCL-Cities/EDGAR-GHG/CO2orgshort')
n2o_ic = ee.ImageCollection('projects/wri-datalab/SCL-Cities/EDGAR-GHG/N2O')

In [ ]:
scale = 100

In [6]:
YEARS = [2000, 2005, 2010, 2015, 2020]

In [ ]:
bycountry = {}
for idx, year in enumerate(YEARS):
    urbext = {}
    urbext_data = {}
    urbext[year] = ee.FeatureCollection('projects/wri-datalab/SCL-Cities/urbanextents__bycountry_{0}'.format(year))

    with open('popcodes.json', 'r') as popfile:
        popcodes2 = json.loads(popfile.read())
    urbext_data[year] = geemap.ee_to_df(urbext[year])

    emi_res = defaultdict(list)
    pop_res = defaultdict(list)

    emi_total = {}
    pop_total = {}

    emi_total[year] = []
    pop_total[year] = []
         
    print()
    print(year)
    ch4 = ch4_ic.filter(ee.Filter.eq('year', year)).first()
    co2_exclshort = co2_exclshort_ic.filter(ee.Filter.eq('year', year)).first()
    co2_short = co2_short_ic.filter(ee.Filter.eq('year', year)).first()
    n2o = n2o_ic.filter(ee.Filter.eq('year', year)).first()
    co2e = (co2_exclshort.add(co2_short).add(n2o.multiply(GWP['N2O'])).add(ch4.multiply(GWP['CH4']))).multiply(31536) # avg tonnes per sq-m per year
    for i in range(len(urbext_data[year])):
        ua = urbext_data[year].iloc[i]
        ua_f = urbext[year].filter(ee.Filter.eq('city_ids', str(ua['city_ids'])))
        geom = ua_f.geometry()
        if ua['country'] in popcodes:
            ghspop_img = ee.ImageCollection("JRC/GHSL/P2023A/GHS_POP").filter(ee.Filter.calendarRange(year, year, 'year')).select('population_count').first()
            pop_img = ghspop_img.divide(ghspop_img.pixelArea())
            
            localpop = ghspop_img.reduceRegion(ee.Reducer.sum(), geom, scale, maxPixels=1e12).get('population_count').getInfo()

            localemi = co2e.multiply(ee.Image.pixelArea()).reduceRegion(ee.Reducer.sum(), geom, scale, maxPixels=1e12).getInfo()['b1']

            emi_res[(ua['country'], year)].append(localemi)
            pop_res[(ua['country'], year)].append(localpop)
            emi_total[year].append(localemi)
            pop_total[year].append(localpop)

        print(i, end=' ')

    if idx == 0:
        bycountry['global'] = {}  
    bycountry['global'] = bycountry['global'] | {
    'urban_ghg_emissions_tonnesCO2e_{0}'.format(year): sum(emi_total[year]),
    'urban_population_{0}'.format(year): sum(pop_total[year]),
    'urban_ghg_percapita_tonnesCO2e_{0}'.format(year): sum(emi_total[year]) / sum(pop_total[year])
    }

    country_list = [i for i in list(set([i[0] for i in list(emi_res.keys())])) if (type(i)==str and len(i)>1)]
    country_list.sort()
    for country in country_list:
        if idx == 0:
            bycountry[country] = {}
          
        bycountry[country] = bycountry[country] | {
                'urban_ghg_emissions_tonnesCO2e_{0}'.format(year): sum(emi_res[(country, year)]),
                'urban_population_{0}'.format(year): sum(pop_res[(country, year)]),
                'urban_ghg_percapita_tonnesCO2e_{0}'.format(year): sum(emi_res[(country, year)]) / sum(pop_res[(country, year)])
            }
        
res_pd = pd.DataFrame.from_dict(bycountry, orient='index')
res_pd.to_csv('CTY-4_GHGemissions_v3.csv')